<a href="https://colab.research.google.com/github/gomezc08/AI_Udemy/blob/main/Deep_Convolutional_Q_Learning_Pac_Man.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!pip install ale-py
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (739 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):
  def __init__(self, action_size, seed = 42):
    super(Network, self).__init__()   # inheritance purposes (inheriting from nn.module so need to ensure we bring in appropriate methods).
    self.seed = torch.manual_seed(seed)
    self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
    self.bn2 = nn.BatchNorm2d(64)
    self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
    self.bn3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
    self.bn4 = nn.BatchNorm2d(128)
    self.fc1 = nn.Linear(10 * 10 * 128, 512)     # 10x10 image with 128 features.
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, action_size)

  # state - images of our initial state.
  def forward(self, state):
    x = F.relu(self.bn1(self.conv1(state)))   # find the conv of the state and then we will normalize it and then activate it.
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))
    x = x.view(x.size(0), -1)   # similar to flatten; RESHAPES TENSOR: takes the size of x from previous line and flattens.
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import ale_py
import gymnasium as gym
# deterministic means the ghost will make deterministic moves (won't move intelligently).
env = gym.make('MsPacmanDeterministic-v0', full_action_space = False)   # only 9 action space vs 18.
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)   # 210 x 160 images with 3 channels; we will resize this in the preprocess frames.
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (210, 160, 3)
State size:  210
Number of actions:  9


/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


### Initializing the hyperparameters

In [ ]:
alpha = 5e-4                # learning rate: how much model updates parameter (gradient descent step size)
batch_size = 64             # updating parameters in batches of 100
gamma = 0.99                # discount factor: how to reward agent; [0,1] -> [immediate reward, future rewards]

### Preprocessing the frames

In [ ]:
# turn the convolution tensors to vector tensors - so it can be fed to network of AI
from PIL import Image
from torchvision import transforms

def preprocess_frame(frame):
  # array to PIL image.
  frame = Image.fromarray(frame)

  # PIL image -> tensor.
  preprocess_object = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])     # working with 128x128 images from pacman.
  return preprocess_object(frame).unsqueeze(0)

### Implementing the DCQN class

In [ ]:
import random

# Agent - defines behaviour of our agent as it interacts with the enviornment.
# act - selects an action based on q-value from network while taking into account epsilon (go through network and get q-value).
# learn - calculates target and expected q-value (using equation) and learns because we wanna minimize our loss function.
# step -  store experiences and decide when to learn from them (every 4 steps).

class Agent():
  # local_qnetwork - the network we are actually doing training on (where we are grabbing our q-values from and teaching agent to move).
  # target_qnetwork - used for computing the loss (actual q-value).

  def __init__(self, action_size):
     self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")       # using gpu.
     self.action_size = action_size
     self.local_qnetwork = Network(action_size).to(self.device)
     self.target_qnetwork = Network(action_size).to(self.device)
     self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = alpha)
     self.memory = deque(maxlen = 1000)

  # store experiences and decide when to learn from them (every 4 steps).
  def step(self, state, action, reward, next_state, done_status):
    # preprocess.
    state = preprocess_frame(state)
    next_state = preprocess_frame(next_state)
    # append experience to memory.
    self.memory.append((state, action, reward, next_state, done_status))
    # ensure we have enough to make batches of memory.
    if len(self.memory) > batch_size:
      experience_batch = random.sample(self.memory, k = batch_size)
      self.learn(experience_batch, gamma)

  # selecting an action (highest q value) based on the current state + epsilon.
  def act(self, state, epsilon = 0.):
    state = preprocess_frame(state).to(self.device)      # adding a batch to the state to show which batch the state belongs to.
    self.local_qnetwork.eval()    # going from training mode -> evaluation mode (kinda like an isoloated evaluation environment).
    # ensure we are using evaluation mode before grabbing our predicted q values.
    with torch.no_grad():
      action_values = self.local_qnetwork(state)

    self.local_qnetwork.train()  # going from evaluation mode -> training mode.
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  # updating q-values from our local network to better match those of the target network.
  """def learn(self, experiences, discount_factor):
    # unpack our experiences.
    states, actions, rewards, next_states, dones = zip(*experiences)

    # Remove extra batch dimension before stacking
    states = torch.cat([s.squeeze(0) for s in states]).float().to(self.device)
    next_states = torch.cat([s.squeeze(0) for s in next_states]).float().to(self.device)

    # Reshape to (batch_size, channels, height, width)
    states = states.view(batch_size, 3, 128, 128)
    next_states = next_states.view(batch_size, 3, 128, 128)

    actions = torch.from_numpy(np.vstack([actions])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([rewards])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([dones])).to(torch.uint8).float().to(self.device)

    Q_target_next = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    Q_target = rewards + (discount_factor * Q_target_next * (1 - dones))    # Q_t equation.
    Q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(Q_expected, Q_target)
    self.optimizer.zero_grad()
    loss.backward()   # back propogates.
    self.optimizer.step()  """                                      # updates local model's parameters.

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = zip(*experiences)
    states = torch.from_numpy(np.vstack(states)).float().to(self.device)
    actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

### Initializing the DCQN agent

In [ ]:
agent = Agent(action_size=number_actions)

### Initialize Hyperprarameters to Training

In [ ]:
epochs = 2000
maximum_number_timesteps_per_episode = 10000    # max number of steps for the rocket to reach surface (dont want AI to train too hard).
# epsilon = exploitation vs exploration; 1 = exploration, 0 = exploitation.
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_100_episodes = deque(maxlen=100)   # window that keeps track of the scores of last 100 episodes.

### Training the DCQN agent

In [ ]:
for epoch in range(1, epochs + 1):
  state, _ = env.reset()    # since we are starting a new episode, we gotta reset the env everytime (reset rocket ship). _ ensures we dont care abt any return value.
  score = 0   # cummulative reward for entire episode (wanna maximize this).
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)   # built in.
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    # check if episode is done or not.
    if done:
      break
  scores_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon * epsilon_decay_value)
  print('\rEpoch {}\tAverage Score: {:.2f}'.format(epoch, np.mean(scores_100_episodes)), end = "")
  if epoch % 100 == 0:
    print('\rEpoch {}\tAverage Score: {:.2f}'.format(epoch, np.mean(scores_100_episodes)), end = "")
    # score for pacman is a score of 500 (coins eaten)
  if np.mean(scores_100_episodes) >= 500.0:
    print('\nEnviornment solved in {:d}\tAverage Score: {:.2f}'.format(epoch, np.mean(scores_100_episodes)), end = "")
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Epoch 84	Average Score: 380.60

## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

NameError: name 'agent' is not defined